---

In [1]:
import pandas as pd

In [3]:
# Leelos los archivos CSV
nombresConId = pd.read_csv('../inventores_con_id.csv')
inventoresPatentes = pd.read_csv('../InventoresPatentes.csv')

In [4]:
# quitamos acentos a los nombres de los inventores en nombresConId en la columna 'Inventor'
nombresConId['Inventor'] = nombresConId['Inventor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [5]:
#Cruzamos los dos dataframes por la columna 'Inventor' y 'Inventor' y solamente nos quedamos con la columna inventor_id del df nombresConId
inventoresPatentes1 = pd.merge(inventoresPatentes, nombresConId[['Inventor', 'inventor_id']], on='Inventor', how='left')

In [7]:
# Eliminamos duplicados conforme al inventores y patentes
inventoresPatentes1 = inventoresPatentes1.drop_duplicates(subset=['Inventor', 'inventor_id'])

In [8]:
# Guardamos el resultado en un nuevo archivo CSV
inventoresPatentes1.to_csv('../datasetPatentes.csv', index=False)

---

In [ ]:
import pandas as pd
import os
import re
# Ruta de la carpeta 'Autores WoS'
carpeta = '../Autores WoS'

# Lista para almacenar los DataFrames procesados
dataframes = []

# Mapa de meses
MESES = {
    'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6,
    'JUL':7, 'AUG':8, 'SEP':9, 'OCT':10, 'NOV':11, 'DEC':12
}

# Definir constantes
SIN_DATOS = 'Sin datos'
TOTAL_CITAS = 'Total de Citas'
PROMEDIO_POR_AÑO = 'Promedio por año'
PIBLICACION_DATE= 'Publication Date'

def calcular_indice_h(df):
    """
    Calcula el índice h para un DataFrame basado en la columna TOTAL_CITAS.
    """
    citas = df[TOTAL_CITAS].sort_values(ascending=False).values
    h_index = 0
    for i, c in enumerate(citas):
        if c >= i + 1:
            h_index = i + 1
        else:
            break
    return h_index

def normaliza_fecha(s):
    if pd.isna(s):
        return pd.NaT

    txt_original = str(s)
    txt = txt_original.strip().upper()
    txt = re.sub(r'\s+', ' ', txt)  # Normaliza espacios
    txt = re.sub(r'^(SPR|SUM|FAL|WIN)(\d{4})$', r'\1 \2', txt)  # WIN1999 → WIN 1999
    txt = re.sub(r'^([A-Z]{3})-[A-Z]{3} (\d{4})$', r'\1 \2', txt)  # APR-JUN 2008 → APR 2008

    season_months = {'SPR': 3, 'SUM': 6, 'FAL': 9, 'WIN': 12}
    MESES = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }

    try:
        if re.fullmatch(r'\d{4}', txt):
            return pd.Timestamp(int(txt), 1, 1)

        # Primero manejar temporadas
        m3 = re.fullmatch(r'(SPR|SUM|FAL|WIN) (\d{4})', txt)
        if m3:
            season, anio = m3.groups()
            return pd.Timestamp(int(anio), season_months[season], 1)

        # A continuación años abreviados
        m = re.fullmatch(r'([A-Z]{3})-(\d{2})', txt)
        if m:
            mon, yy = m.groups()
            return pd.Timestamp(2000 + int(yy), MESES[mon], 1)

        # Solo procesar meses válidos
        m2 = re.fullmatch(r'([A-Z]{3}) (\d{4})', txt)
        if m2:
            mon, anio = m2.groups()
            if mon in MESES:
                return pd.Timestamp(int(anio), MESES[mon], 1)

        # Si es solo temporada sin año
        if txt in season_months:
            hoy = pd.Timestamp.now()
            return pd.Timestamp(hoy.year, season_months[txt], 1)

        return pd.to_datetime(txt, errors='raise')

    except Exception as e:
        print(f"[ERROR] Fecha no convertida: '{txt_original}' (procesado como: '{txt}') → {e}")
        return pd.NaT

for archivo in os.listdir(carpeta):
    if archivo.endswith('.txt'):
        try:
            # Leer el archivo y eliminar las primeras 3 filas
            df = pd.read_csv(
                os.path.join(carpeta, archivo),
                skiprows=3,
                on_bad_lines='skip',
                engine='python'
            )

            # Agregar la columna "Investigador"
            df['Investigador'] = os.path.splitext(archivo)[0]

            # contamos cuantos autores hay en la columna 'Authors' separados po ;
            df['Num_Autores'] = df['Authors'].str.split(';').str.len()

            # Normalizamos la columna 'Publicarion Date'
            df[PIBLICACION_DATE] = df[PIBLICACION_DATE]\
                .apply(normaliza_fecha)\
                .dt.date
            
            # Si PIBLICACION_DATE es NA vemos que año hay en 'Publication Year' y lo pasamos a fecha en formato YYYY-MM-DD
            df[PIBLICACION_DATE] = df.apply(
                lambda x: pd.Timestamp(f"{x['Publication Year']}-01-01").date() 
                if pd.isna(x[PIBLICACION_DATE]) else x[PIBLICACION_DATE], 
                axis=1
            )

            # Eliminar sólo las columnas no deseadas
            df = df.drop(columns=['Average per Year', 'Total Citations'], errors='ignore')

            # Detectar columnas de años 2000–2024
            columnas_años = [str(año) for año in range(2000, 2024)]
            años_existentes = [c for c in columnas_años if c in df.columns]

            # Eliminar columnas de años 1900–1999 y la del 2025
            columnas_a_eliminar = [str(año) for año in range(1900, 2000)]
            df = df.drop(columns=columnas_a_eliminar, errors='ignore')
            df = df.drop(columns=['2025'], errors='ignore')

            df[TOTAL_CITAS] = df[años_existentes].sum(axis=1)
            df[PROMEDIO_POR_AÑO] = df['Total de Citas'] / ((2024 - df['Publication Year']) + 1)

            # 'Promedio por año' lo dejamos a 2 digitos
            df[PROMEDIO_POR_AÑO] = df[PROMEDIO_POR_AÑO].round(2)

            # Calcular el índice h para el investigador
            df['h'] = calcular_indice_h(df)

            dataframes.append(df)

        except Exception as e:
            print(f"Error al procesar {archivo}: {e}")

# Concatenar y exportar
df_combinado = pd.concat(dataframes, ignore_index=True)

df_combinado[TOTAL_CITAS] = df_combinado[TOTAL_CITAS].fillna(0)
df_combinado[PROMEDIO_POR_AÑO] = df_combinado[PROMEDIO_POR_AÑO].fillna(0)
df_combinado.replace([float('inf'), float('-inf')], float('nan'), inplace=True)

# En estas columnas 'Title', 'Investigador', 'Corporate Authors', 'Book Editors', 'Source Title' si hay NaN los convertimos a 'Sin datos'
df_combinado['Title'] = df_combinado['Title'].fillna(SIN_DATOS)
df_combinado['Investigador'] = df_combinado['Investigador'].fillna(SIN_DATOS)
df_combinado['Corporate Authors'] = df_combinado['Corporate Authors'].fillna(SIN_DATOS)
df_combinado['Book Editors'] = df_combinado['Book Editors'].fillna(SIN_DATOS)
df_combinado['Source Title'] = df_combinado['Source Title'].fillna(SIN_DATOS)

# Guardamos el resultado en un nuevo archivo CSV
df_combinado.to_csv('../Analisis/datasetWoS.csv', index=False, encoding='utf-8')

print(f"Concatenados {len(dataframes)} archivos en 'dataset_autores_wos.csv'.")

Concatenados 190 archivos en 'dataset_autores_wos.csv'.


In [24]:
# Enumerar las columnas con sus índices
for i, col in enumerate(df_combinado.columns):
    print(f"{col}")

Title
Authors
Corporate Authors
Editors
Book Editors
Source Title
Publication Date
Publication Year
Volume
Issue
Part Number
Supplement
Special Issue
Beginning Page
Ending Page
Article Number
DOI
Conference Title
Conference Date
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
Investigador
Num_Autores
Total de Citas
Promedio por año
h


In [11]:
# Verificar si hay valores NaN en las columnas
if df_combinado.isnull().any().any():
    print("Hay valores NaN en el DataFrame.")
    print(df_combinado.isnull().sum())  # Mostrar el conteo de NaN por columna

# Verificar si hay valores infinitos en las columnas
if (df_combinado == float('inf')).any().any() or (df_combinado == float('-inf')).any().any():
    print("Hay valores infinitos en el DataFrame.")
    print(df_combinado[(df_combinado == float('inf')) | (df_combinado == float('-inf'))])  # Mostrar las filas con valores infinitos

Hay valores NaN en el DataFrame.
Title                   0
Authors                 0
Corporate Authors       0
Editors              4063
Book Editors            0
Source Title            0
Publication Date       16
Publication Year        0
Volume                286
Issue                1265
Part Number          3979
Supplement           3968
Special Issue        3919
Beginning Page        773
Ending Page           773
Article Number       3351
DOI                   559
Conference Title     3442
Conference Date      3442
2000                    0
2001                    0
2002                    0
2003                    0
2004                    0
2005                    0
2006                    0
2007                    0
2008                    0
2009                    0
2010                    0
2011                    0
2012                    0
2013                    0
2014                    0
2015                    0
2016                    0
2017                    0
2018 

In [12]:
# Leemos cada txt de la carpeta 'Autores WoS' y los guardamos en una nueva carpeta llamada 'Autores WoS Limpios' donde solamente se le quiten las 3 primeras filas
carpeta_limpia = '../Autores WoS Limpios'
os.makedirs(carpeta_limpia, exist_ok=True)
for archivo in os.listdir(carpeta):
    if archivo.endswith('.txt'):
        try:
            # Leer el archivo y eliminar las primeras 3 filas
            df = pd.read_csv(
                os.path.join(carpeta, archivo),
                skiprows=3,
                on_bad_lines='skip',
                engine='python'
            )

            # Guardar el DataFrame limpio en la nueva carpeta
            df.to_csv(os.path.join(carpeta_limpia, archivo), index=False, encoding='utf-8')

        except Exception as e:
            print(f"Error al procesar {archivo}: {e}")

# Verificar si la carpeta 'Autores WoS Limpios' se creó correctamente
if os.path.exists(carpeta_limpia):
    print(f"La carpeta '{carpeta_limpia}' se creó correctamente.")

La carpeta '../Autores WoS Limpios' se creó correctamente.


---

In [7]:
import pandas as pd

df = pd.read_csv('../Analisis/Nombres_PxS.csv')

# Hacemos que la columna 'NOMBRE SNII' tenga formato de 'NOMBRE DEL INVESTIGADOR'
df['NOMBRE SNII'] = df['NOMBRE DEL INVESTIGADOR'].str.replace(',', ' ', regex=False)

# Eliminamos la columna Ratio, NOMBRE PATENTE
df = df.drop(columns=['Ratio', 'NOMBRE PATENTE'], errors='ignore')

# guardamos el resultado en un nuevo archivo CSV
df.to_csv('../Analisis/Nombres_PxS.csv', index=False, encoding='utf-8')